In [ ]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

In [ ]:
MESSAGE = "clean file & gitignore again"
!git config --global user.email "ronyginosar@mail.huji.ac.il"
!git config --global user.name "ronyginosar"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

Empty

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from PIL import Image
import torch
import torchvision.transforms as transforms
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches

dataset_root = ''

# 1. Implement the custom Dataset
class CityPersons(Dataset):
    def __init__(self, dataset_root, split, ...):
        assert split in ['train','val','test']
        ...

    def __len__(self):
        return ...

    def __getitem__(self, idx):
        ...
        return img, seg

    # visualize a sample of the dataset (index idx)
    def show(self, idx):
        ...

# 2. Create an instance of the dataset and verify by visualization
dataset = CityPersons(...)

# 3. Write a custom collate function
def my_collate(samples_list):
    ...
    return batch

# 4. Create a DataLoader using the custom collate function
dataloader = DataLoader(dataset, ...)

# 5. Generate a batch and verify correctness


# 6. Add an augmentation for random horizontal flip (both image & GT)
#  Verify by visualization

Solution

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from PIL import Image
import torch
import torchvision.transforms as transforms
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class CityPersons(Dataset):
    def __init__(self, root_dir, split):
        assert split in ['train','val','test']
        self.root_dir = root_dir
        self.split = split
        img_tag, ann_tag = "leftImg8bit", "gtBboxCityPersons"
        self.images = sorted(list((Path(root_dir)/"leftImg8bit"/split).rglob("*.png")))
        self.annotations =[Path(root_dir)/ann_tag/split/path.parent.name/
                           path.name.replace(f"_{img_tag}.png",f"_{ann_tag}.json")
                           for path in self.images]
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Get the image
        img = Image.open(self.images[idx])
        img = transforms.ToTensor()(img)
        # Get the GT
        bbox_coords = torch.empty(0,4)
        bbox_lbl = []
        if self.split!='test':
            with open(self.annotations[idx]) as f:
                anns = json.load(f)
            if anns['objects']:
                bbox_coords = torch.stack([torch.FloatTensor(ann['bboxVis']) for ann in anns['objects']])
                bbox_lbl = [ann['label'] for ann in anns['objects']]
        sample = {
            'image': img,
            'bbox_coords': bbox_coords,
            'bbox_lbl': bbox_lbl
        }
        return sample
        #return img, bbox_coords, bbox_lbl

    def show(self,idx):
        sample = self[idx]
        fig, ax = plt.subplots()
        ax.imshow(sample['image'].permute((1,2,0)).numpy())
        for bbox, lbl in zip(sample['bbox_coords'],sample['bbox_lbl']):
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            plt.text(bbox[0], bbox[1]-5, lbl,color='yellow')
        plt.title(f'CityPersons annotations for "{self.images[idx].relative_to(self.root_dir)}"')
        plt.show()

def my_collate(samples_list):
    # Assumes samples_list is a list of dicts; All dicts must have same keys
    batch = {}
    for key, val in samples_list[0].items():
        list_of_vals = [sample[key] for sample in samples_list]
        if torch.is_tensor(val) and val.ndim==3:
            batch[key] = torch.stack(list_of_vals)
        else:
            batch[key] = list_of_vals
    return batch

root_dir = '' # add path cityscapes 
dataset = CityPersons(root_dir, 'val')
sample = dataset[344]

dataloader = DataLoader(dataset, batch_size=4, num_workers=0, shuffle=True,
                        collate_fn=my_collate)

d_iter = iter(dataloader)
batch = d_iter.next()

for batch in dataloader:
    print(batch)